In [ ]:
import requests
import pandas as pd
import os

# Defina sua chave API do FRED
api_key = '1bcbbf6cffa7aa308484d98a6078c54d'

# Defina o caminho do arquivo CSV de entrada
input_csv_path = 'xxxx'

# Verificar se o arquivo CSV existe
if not os.path.exists(input_csv_path):
    print(f'Arquivo CSV não encontrado em: {input_csv_path}')
    exit()

# Carregar o arquivo CSV para um DataFrame, especificando o separador como ponto e vírgula e o encoding como latin1
print(f'Lendo arquivo CSV de entrada: {input_csv_path}')
df_indicadores = pd.read_csv(input_csv_path, sep=';', encoding='latin1')

# URL base da API do FRED
base_url = 'https://api.stlouisfed.org/fred/series/observations'

# Função para buscar metadados do FRED
def fetch_metadata(series_id):
    url = f'https://api.stlouisfed.org/fred/series?series_id={series_id}&api_key={api_key}&file_type=json'
    print(f'Fetching metadata from: {url}')  # Imprimir URL para verificação
    response = requests.get(url)
    print(f'Status Code: {response.status_code}')  # Imprimir status da resposta
    if response.status_code == 200:
        try:
            metadata = response.json()
            return metadata
        except ValueError as e:
            print(f'Error decoding JSON for {series_id}: {e}. Response content: {response.text}')  # Imprimir conteúdo da resposta em caso de erro
    else:
        print(f'Failed to fetch metadata for {series_id}. Status Code: {response.status_code}')
    return None

# Função para buscar dados do FRED
def fetch_data(series_id, retries=3):
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json&observation_start=2000-01-01'
    print(f'Fetching data from: {url}')  # Imprimir URL para verificação
    
    for attempt in range(retries):
        response = requests.get(url)
        print(f'Status Code: {response.status_code}')  # Imprimir status da resposta
        
        if response.status_code == 200:
            try:
                data = response.json()
                
                # Buscar metadados da série para obter o nome completo do indicador
                metadata_url = f'https://api.stlouisfed.org/fred/series?series_id={series_id}&api_key={api_key}&file_type=json'
                print(f'Fetching metadata from: {metadata_url}')  # Imprimir URL para verificação
                metadata_response = requests.get(metadata_url)
                print(f'Metadata Status Code: {metadata_response.status_code}')  # Imprimir status da resposta
                
                if metadata_response.status_code == 200:
                    metadata = metadata_response.json()
                    if 'seriess' in metadata and len(metadata['seriess']) > 0:
                        name = metadata['seriess'][0].get('title', '')  # Obtendo o nome completo do indicador
                        unit = metadata['seriess'][0].get('units', '')
                        frequency = metadata['seriess'][0].get('frequency', '')
                        source = metadata['seriess'][0].get('source', '')
                        api = metadata['seriess'][0].get('api', '')
                        
                        data['name'] = name
                        data['unit'] = unit
                        data['frequency'] = frequency
                        data['source'] = source
                        data['api'] = api
                        
                        return data
                    else:
                        print(f'Failed to fetch metadata for {series_id}. Metadata response: {metadata_response.text}')
                else:
                    print(f'Failed to fetch metadata for {series_id}. Metadata Status Code: {metadata_response.status_code}')
            except ValueError as e:
                print(f'Error decoding JSON for {series_id}: {e}. Response content: {response.text}')  # Imprimir conteúdo da resposta em caso de erro
            break  # Saia do loop se os dados foram processados corretamente
        else:
            print(f'Failed to fetch data for {series_id}. Status Code: {response.status_code}')
            if attempt < retries - 1:  # Se não for a última tentativa
                print(f'Tentando novamente... (Tentativa {attempt + 2}/{retries})')
            else:
                print(f'Falhou após {retries} tentativas. Indo para o próximo indicador.')
    
    return None

# Buscar dados para cada indicador listado no arquivo CSV
data_frames = []

for index, row in df_indicadores.iterrows():
    code = row['codigo']
    
    # Buscar dados do FRED para o indicador atual
    print(f'Buscando dados para o indicador: {code}')
    data = fetch_data(code)
    
    if data and 'observations' in data:
        df = pd.DataFrame(data['observations'])
        
        # Adicionar metadados ao DataFrame
        df['codigo'] = code
        df['nome_completo'] = data.get('name', '')  # Adicionar o nome completo do indicador
        
        data_frames.append(df)
    else:
        print(f'Nenhum dado encontrado para o indicador: {code}')

# Verificar se algum dado foi coletado
if data_frames:
    # Concatenar todos os DataFrames
    final_df = pd.concat(data_frames, ignore_index=True)
    
    # Remover a coluna 'realtime_end' se existir
    if 'realtime_end' in final_df.columns:
        final_df.drop(columns=['realtime_end'], inplace=True)
    
    # Remover a coluna 'realtime_start' se existir
    if 'realtime_start' in final_df.columns:
        final_df.drop(columns=['realtime_start'], inplace=True)
    
    # Renomear a coluna 'date' para 'data' e formatar
    if 'date' in final_df.columns:
        final_df['data'] = pd.to_datetime(final_df['date']).dt.strftime('%d/%m/%Y')
        final_df.drop(columns=['date'], inplace=True)  # Remover a coluna original
    
    # Renomear a coluna 'value' para 'valor'
    if 'value' in final_df.columns:
        final_df.rename(columns={'value': 'valor'}, inplace=True)
    
    # Excluir linhas onde 'valor' é igual a 0 ou ","
    final_df = final_df[(final_df['valor'] != '0') & (final_df['valor'] != ',')]
    
    # Criar a coluna 'fonte' com valor 'FRED'
    final_df['fonte'] = 'FRED'
    
    # Excluir linhas onde 'valor' é igual a ","
    final_df = final_df[final_df['valor'] != ',']

    # Substituir "." por "," nas colunas numéricas e formatar unidades
    def format_decimal(value):
        try:
            return str(value).replace('.', ',') if pd.notnull(value) else value
        except Exception as e:
            print(f'Error formatting value {value}: {e}')
            return value
    
    final_df = final_df.applymap(format_decimal)

    # Modificar as colunas "codigo" e "nome_completo"
    final_df['codigo'] = 'FRED - ' + final_df['codigo']
    final_df['nome_completo'] = final_df['codigo'] + ' - ' + final_df['nome_completo']
    
    # Salvar taxas em um CSV separado
    taxas_csv_path = 'xxxx'
    final_df.to_csv(taxas_csv_path, index=False, sep=';')  # Salvar usando ponto e vírgula como separador
    print(f'Taxas dos indicadores salvas em: {taxas_csv_path}')
else:
    print('Nenhum dado foi coletado.')
    
import pandas as pd
import os

# Defina o caminho do arquivo CSV de saída
csv_path = 'xxxx'

# Verificar se o arquivo CSV existe
if not os.path.exists(csv_path):
    print(f'Arquivo CSV não encontrado em: {csv_path}')
    exit()

# Carregar o arquivo CSV para um DataFrame
print(f'Lendo arquivo CSV de entrada: {csv_path}')
df = pd.read_csv(csv_path, sep=';', encoding='latin1')

# Remover linhas onde 'valor' é exatamente ','
initial_count = len(df)
df = df[df['valor'] != ',']
removed_count = initial_count - len(df)

# Salvar o DataFrame modificado no mesmo arquivo CSV
df.to_csv(csv_path, index=False, sep=';')
print(f'Dados salvos em: {csv_path}')
print(f'{removed_count} linhas removidas onde "valor" era ",".')


import pandas as pd
import os
from datetime import datetime

# Caminho do arquivo original
caminho_arquivo_original = r"xxxx"

# Lê o arquivo CSV original
df = pd.read_csv(caminho_arquivo_original, sep=';')

# Imprime os nomes das colunas para verificar
print("Colunas disponíveis:", df.columns.tolist())

# Remove espaços em branco dos nomes das colunas
df.columns = df.columns.str.strip()

# Verifica o tipo da coluna 'date'
print("Tipos de dados das colunas:")
print(df.dtypes)

# Converte a coluna 'date' para o tipo datetime, se necessário
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Extrai a data de atualização da base (data atual)
data_atualizacao_base = datetime.now().strftime('%Y-%m-%d')

# Processa os dados para obter a data mais recente para cada código
resultado = df.groupby(['codigo', 'nome_completo']).agg(
    Data_do_ultimo_dado=('data', 'max')
).reset_index()

# Adiciona a data de atualização da base
resultado['Data_de_atualizacao_da_base'] = data_atualizacao_base

# Define o caminho do novo arquivo CSV
caminho_novo_arquivo = os.path.join(
    os.path.dirname(caminho_arquivo_original), 
    'FRED_resumo.csv'
)

# Salva o novo arquivo CSV
resultado.to_csv(caminho_novo_arquivo, index=False,sep=";")

print(f'Arquivo salvo em: {caminho_novo_arquivo}')


Lendo arquivo CSV de entrada: C:/Users/DXX7/PETROBRAS/Integração de Dados - SUB ORC - Documentos/INDICES MACROECONOMICOS/Scripts/Scripts/FRED/Dados de entrada/FRED_entrada.csv
Buscando dados para o indicador: PPIACO
Fetching data from: https://api.stlouisfed.org/fred/series/observations?series_id=PPIACO&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json&observation_start=2000-01-01
Status Code: 200
Fetching metadata from: https://api.stlouisfed.org/fred/series?series_id=PPIACO&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json
Metadata Status Code: 200
Buscando dados para o indicador: WPU01PLUS02
Fetching data from: https://api.stlouisfed.org/fred/series/observations?series_id=WPU01PLUS02&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json&observation_start=2000-01-01
Status Code: 200
Fetching metadata from: https://api.stlouisfed.org/fred/series?series_id=WPU01PLUS02&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json
Metadata Status Code: 200
Buscando dados para o i

C:\Users\DXX7\AppData\Local\Temp\ipykernel_36064\367990364.py:150: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df = final_df.applymap(format_decimal)


OSError: Cannot save file into a non-existent directory: 'C:\Users\DXX7\PETROBRAS\Integração de Dados - SUB ORC - Scripts\Scripts\FRED\Dados de saida'